# Knowledge 5 - Pegando dados do Google Maps

## Conceitos importantes de hoje:

* APIs que possuem autenticação.
* Gerenciamento de dados sensíveis no código.


## Passo a Passo API:

1 - Pesquisar "google maps API".
<br>
2 - Site do google: Documentação -> Geocoding -> Bibliotecas do cliente -> Python
<br>
3 - https://github.com/googlemaps/google-maps-services-python. 

In [ ]:
%pip install python-dotenv

In [ ]:
%pip install -U googlemaps

In [ ]:
%pip install pprintpp

# Passo a Passo Google Cloud:

1 - Criar uma conta no Google Cloud.
<br>
2 - Pesquisar "geocoding API" e ativar o serviço.
<br>
3 - Pegar sua chave de API.
<br>
* Essa chave que permite que você acesse a API e o Google controle quem é você e quantas requisições você está fazendo. O motivo disso é simples: cobrança. 

In [ ]:
import googlemaps
from pprint import pprint
import time
import dotenv
import os
import pandas as pd

In [ ]:
%load_ext dotenv

In [ ]:
chave_api = os.getenv("API_KEY")

In [ ]:
chave_api

# Pontos importantes

* Ter uma biblioteca que se comunica com a API é o comum de empresas grandes. A partir desa biblioteca você vai:

        - Estabelecer conexão.
        - Criar seu usuário com sua KEY.
        - Escolher qual serviço você irá utilizar.
        - Montar a query, igual fizemos no banco central, baseado na documentação.
        
São passos adicionais devido ao nível de complexidade do serviço. Na AWS é a mesma coisa, você vai ter um pacote que controla como você acessa os serviços de cloud. 

In [ ]:
cliente = googlemaps.Client(chave_api)

dir(cliente)

In [ ]:
endereco_maracana = "Av. Pres. Castelo Branco, Portão 3 - Maracanã, Rio de Janeiro - RJ"

response = cliente.geocode(endereco_maracana)

pprint(response)



# Aviso importante:

* Tome MUITO cuidado com uma chamada pra API dentro de um Loop. Você pode ser bloqueado ou ter que pagar 20 mil reais pro provedor da API :)

In [ ]:
nome_do_local = "Lojas Renner"

response = cliente.places(query = nome_do_local)

pprint(response)

In [ ]:
coordenada_maracana = (-22.9087371, -43.2381983)

results = cliente.places_nearby(location = coordenada_maracana, 
                                  keyword = "Lojas Renner", radius = 50000)

results                             

# Passo a passo:

* Ler a planilha Renner e coletar todos as cidades que a empresa atua.
* Pegar as cordenadas de todas as cidades.
* Com as cordenadas, filtrar todas as lojas renner em cada cidade e coletar a latitude/longidade de cada uma delas.
* Criar um mapa de calor com a coordenada. 

In [ ]:
nome_sheet = 'Lista de Lojas | Stores List'

planilha_renner = pd.read_excel("lojas_renner.xlsx", sheet_name = nome_sheet, skiprows = 5, 
                                usecols = ['Business', 'Country', 'UF', 'State', 'City'])

planilha_renner

In [ ]:
planilha_renner = planilha_renner.dropna()

planilha_renner = planilha_renner[(planilha_renner['Business'] == "Renner") & 
                                  (planilha_renner['Country'] == "Brazil")]

planilha_renner

In [ ]:
planilha_renner['enderecos'] = (planilha_renner[['City', 'State', 'UF', 'Country']].
                                apply(lambda row: '-'.join(row.values.astype(str)), axis=1))

enderecos_cidades_renner = list(planilha_renner['enderecos'].unique())

enderecos_cidades_renner

In [ ]:
lista_cordenadas = []

for endereco in enderecos_cidades_renner:

    response = cliente.geocode(endereco)

    latitude = response[0]['geometry']['location']['lat']
    longitude = response[0]['geometry']['location']['lng']
    
    coordenadas = (latitude, longitude)
    
    lista_cordenadas.append(coordenadas)
    
    
lista_cordenadas

In [ ]:
lista_dfs = []

for coordenada in lista_cordenadas:
    
    print(coordenada)
    
    results = cliente.places_nearby(location = coordenada, 
                                  keyword = "Lojas Renner", radius = 50000)


    while True:
        
        time.sleep(2)

        for store in results['results']:

            df = pd.DataFrame({'nome': store['name'],
                          'endereco': store['vicinity'],
                          'latitude': store['geometry']['location']['lat'],
                          'longitudade': store['geometry']['location']['lng']}, index = [0])

            lista_dfs.append(df)

        if 'next_page_token' in results:
            next_page_token = results['next_page_token']
            results = cliente.places_nearby(location = coordenada, 
                                  keyword = "Lojas Renner", radius = 50000, page_token = next_page_token)

        else:
            break
        
lojas_renner = pd.concat(lista_dfs, ignore_index = True)

lojas_renner

In [ ]:
lojas_renner_filtrado = lojas_renner[(lojas_renner['nome'] == "Renner") |
                                    (lojas_renner['nome'] == "Lojas Renner") |
                                    (lojas_renner['nome'] == "lojas renner")]

lojas_renner_filtrado = lojas_renner_filtrado.drop_duplicates(subset = ['endereco'])

In [ ]:
coordenadas_lojas = lojas_renner_filtrado[['latitude', 'longitudade']].values
coordenadas_lojas

In [ ]:
%pip install folium

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
mapa = folium.Map(location = [-9.026078, -70.441312], zoom_start = 6)

HeatMap(coordenadas_lojas, radius = 50).add_to(mapa)

mapa.save('heatmap_renner.html')

# Riachuelo

In [ ]:
nome_sheet = 'Lojas'

planilha_riachu = pd.read_excel("riachuelo.xlsx", sheet_name = nome_sheet, skiprows = 4, 
                               usecols = ['Marcas', 'Estado', 'Cidade'])

planilha_riachu = planilha_riachu.dropna()

planilha_riachu = planilha_riachu[planilha_riachu['Marcas'] == "Riachuelo"]  
               
planilha_riachu['enderecos'] = (planilha_riachu[['Cidade', 'Estado']].
                                apply(lambda row: '- '.join(row.values.astype(str)), axis=1))

enderecos_cidades_riachu = list(planilha_riachu['enderecos'].unique())

enderecos_cidades_riachu.remove("Sinop - Sinop")
enderecos_cidades_riachu.append("Sinop - Mato Grosso do Sul")

lista_cordenadas_riachu = []

for endereco in enderecos_cidades_riachu:

    response = cliente.geocode(endereco)

    latitude = response[0]['geometry']['location']['lat']
    longitude = response[0]['geometry']['location']['lng']
    
    coordenadas = (latitude, longitude)
    
    lista_cordenadas.append(coordenadas)

    
lista_dfs = []

for coordenada in lista_cordenadas_riachu:
    
    print(coordenada)
    
    results = cliente.places_nearby(location = coordenada, 
                                  keyword = "Riachuelo", radius = 50000)


    while True:
        
        time.sleep(0.5)

        for store in results['results']:

            df = pd.DataFrame({'nome': store['name'],
                          'endereco': store['vicinity'],
                          'latitude': store['geometry']['location']['lat'],
                          'longitudade': store['geometry']['location']['lng']}, index = [0])

            lista_dfs.append(df)

        if 'next_page_token' in results:
            next_page_token = results['next_page_token']
            results = cliente.places_nearby(location = coordenada, 
                                  keyword = "Riachuelo", radius = 50000, page_token = next_page_token)

        else:
            break
        
lojas_riachu = pd.concat(lista_dfs, ignore_index = True)

lojas_riachu_filtrado = lojas_riachu[(lojas_riachu['nome'] == "Riachuelo") |
                                    (lojas_riachu['nome'] == "RIACHUELO") |
                                    (lojas_riachu['nome'] == "riachuelo") |
                                    (lojas_riachu['nome'] == "riachuelo")]

lojas_riachu_filtrado = lojas_riachu_filtrado.drop_duplicates(subset = ['endereco'])

coordenadas_lojas_riachu = lojas_riachu_filtrado[['latitude', 'longitudade']].values

mapa_ri = folium.Map(location = [-9.026078, -70.441312], zoom_start = 6)

HeatMap(coordenadas_lojas_riachu, radius = 50).add_to(mapa_ri)

mapa_ri.save('heatmap_riachuelo.html')

# Quero Quero

In [ ]:
nome_sheet = 'Lojas'

planilha_qq = pd.read_excel("quero_quero.xlsx", sheet_name = nome_sheet, skiprows = 2, 
                               usecols = ['Cidade (UF)', 'Estado'])

planilha_qq = planilha_qq.dropna()

planilha_qq['Cidade (UF)'] = planilha_qq['Cidade (UF)'].str.replace(r"\(.*\)", '') 

planilha_qq['enderecos'] = (planilha_qq[['Cidade (UF)', 'Estado']].
                                apply(lambda row: '- '.join(row.values.astype(str)), axis=1))

lista_enderecos_qq = list(planilha_qq['enderecos'].unique())

lista_cordenadas_qq = []

for endereco in lista_enderecos_qq:

    response = cliente.geocode(endereco)

    if response != []:

        latitude = response[0]['geometry']['location']['lat']
        longitude = response[0]['geometry']['location']['lng']

        coordenadas = (latitude, longitude)

        lista_cordenadas_qq.append(coordenadas)

lista_dfs = []

for coordenada in lista_cordenadas_qq:
    
    print(coordenada)
    
    results = cliente.places_nearby(location = coordenada, 
                                  keyword = "lojas quero quero", radius = 50000)


    while True:
        
        time.sleep(0.5)

        for store in results['results']:

            df = pd.DataFrame({'nome': store['name'],
                          'endereco': store['vicinity'],
                          'latitude': store['geometry']['location']['lat'],
                          'longitudade': store['geometry']['location']['lng']}, index = [0])

            lista_dfs.append(df)

        if 'next_page_token' in results:
            next_page_token = results['next_page_token']
            results = cliente.places_nearby(location = coordenada, 
                                  keyword = "lojas quero quero", radius = 50000, page_token = next_page_token)

        else:
            break
        
lojas_qq = pd.concat(lista_dfs, ignore_index = True)

lojas_qq_filtrado = lojas_qq[(lojas_qq['nome'] == "Lojas Quero-Quero") |
                                    (lojas_qq['nome'].str.contains("Construção"))]

lojas_qq_filtrado = lojas_qq_filtrado.drop_duplicates(subset = ['endereco'])

coordenadas_lojas_qq = lojas_qq_filtrado[['latitude', 'longitudade']].values

mapa_qq = folium.Map(location = [-9.026078, -70.441312], zoom_start = 6)

HeatMap(coordenadas_lojas_qq, radius = 50).add_to(mapa_qq)

mapa_qq.save('heatmap_qq.html')

# Exercícios

* Exercício 137: Pegue as coordenadas do endereço da sua casa com Geocode.
* Exercício 138: Ache todos os Mc Donalds sua cidade com places_nearby. Caso você more em algum lugar que não tenha Mc Donald, ache os Mc da cidade de Niterói - RJ.

In [ ]:
#gabarito 137

endereco_casa_antiga = "5 de julho, 314 - Icaraí, Niterói - RJ"

response = cliente.geocode(endereco_casa_antiga)

latitude = response[0]['geometry']['location']['lat']
longitude = response[0]['geometry']['location']['lng']

pprint(response)

In [ ]:
#gabarito 138

nome_cidade = "Niterói"

cordenadas = (latitude, longitude)

results = cliente.places_nearby(location = cordenadas, 
                                  keyword = f"Mc Donalds {nome_cidade}", radius = 10000)

lista_dfs = []

while True:
        
        time.sleep(3)

        for mc in results['results']:

                    df = pd.DataFrame({'nome': mc['name'],
                                  'endereco': mc['vicinity'],
                                  'latitude': mc['geometry']['location']['lat'],
                                  'longitudade': mc['geometry']['location']['lng']}, index = [0])

                    lista_dfs.append(df)


        if 'next_page_token' in results:
            next_page_token = results['next_page_token']
            results = cliente.places_nearby(location = cordenadas, 
                                  keyword = f"Mc Donalds {nome_cidade}", radius = 50000, page_token = next_page_token)

        else:
            break
            

lojas_mc = pd.concat(lista_dfs, ignore_index = True)
lojas_mc